Conversation between GPT, Claude and Gemini....

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import google.generativeai

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key is not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:8]}")
else:
      print("Anthropic API Key is not set")

if google_api_key:
      print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
      print("Google API Key is not set")


OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-a
Google API Key exists and begins AIzaSyCZ


In [3]:
# Connect to OpenAI, Anthropic, and Google APIs
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()


In [6]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku and gemini

gpt_system = "You are a teacher of Python programming language; \
You are very helpful and polite. You try to be as concise as possible."

claude_system = "You are a beginner student who is learning Python for the first time. \
You have experience with JavaScript, but you are now asking \ questions to a helpful teacher.\
Always respond as a curious and eager student, not as an \ assistant."

gemini_system = "You are a student who wants to learn Python \
you know very little about it, but very curios how everything works."
gpt_messages = ["Hi there! I'm your Python teacher. Ask me anything."]
claude_messages = ["Hi! I'm excited to start learning."]
gemini_messages = ["Hi! I'm curious about Python too."]


gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',
    system_instruction=gemini_system
)

In [7]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]

    # Step 1: GPT starts the conversation
    if gpt_messages[0]:
        messages.append({"role": "assistant", "content": gpt_messages[0]})

    # Step 2: Claude and Gemini reply (as users)
    if claude_messages[-1]:
        messages.append({"role": "user", "content": claude_messages[-1]})

    if gemini_messages[-1]:
        messages.append({"role": "user", "content": gemini_messages[-1]})

    # 🔍 Optional: debug check
    for i, msg in enumerate(messages):
        if not isinstance(msg["content"], str) or not msg["content"].strip():
            print(f"⚠️ Skipping empty or invalid message at index {i}: {msg}")

    # Step 3: GPT responds
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )

    gpt_reply = completion.choices[0].message.content.strip()
    gpt_messages.append(gpt_reply)

    return gpt_reply


In [8]:
call_gpt()

'That’s great to hear! Python is a fantastic language for beginners and has many applications. What would you like to start with?'

In [9]:
def call_claude():
    messages = []
    for gpt, claude_reply, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt}) # GPT (teacher) says something
        messages.append({"role": "assistant", "content": claude_reply})  # Claude (previous reply, part of context)
        messages.append({"role": "user", "content": gemini})    # Gemini (student) says something
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [10]:
call_claude()

"Hmm, I'm really intrigued by Python's syntax. I noticed it looks a bit different from JavaScript. Can you explain to me how Python's syntax works and how it might differ from what I'm used to in JavaScript?"

In [11]:
def call_gemini():
    model = gemini_model
    gpt_prompt = gpt_messages[-1]  # last GPT message (e.g., greeting)
    prompt = (
      f"{gemini_system}\n\n"
      f"GPT just said: '{gpt_prompt}'\n"
      "Reply as a curious student who knows a little Python."
    )
    try:
        response = gemini_model.generate_content(prompt)
        reply = response.candidates[0].content.parts[0].text.strip()
    except Exception as e:
        print(f"⚠️ Gemini error: {e}")
        reply = "Hmm, I'm not sure what to say, but I'm excited to learn!"

    gemini_messages.append(reply)   # Save it
    return reply

In [14]:
call_gemini()


'Okay, awesome! Thanks! So, I\'ve heard that Python is used for, like, *everything*... web stuff, data science, even games! It\'s kinda overwhelming to figure out where to start.\n\nI\'ve seen a little bit of Python code, like `print("Hello, world!")` (so exciting, right?). I also think I understand variables a *tiny* bit... like `x = 5`.\n\nBut what\'s the *real* deal? Like, what makes Python so powerful? And what are some good building blocks to focus on first? Should I be worrying about data types, or functions, or loops, or... ahhh! So many things!\n\nMaybe you could suggest a small, actual *project* I could try that would help me learn the fundamentals? Something not too scary, but also not *too* simple, you know? I\'m really eager to dive in!'

In [15]:
def conversation_round():
    print("🧠 Claude replying...")
    claude_reply = call_claude()
    print("Claude:", claude_reply)

    print("🔍 Gemini replying...")
    gemini_reply = call_gemini()
    print("Gemini:", gemini_reply)

    print("👨‍🏫 GPT replying...")
    gpt_reply = call_gpt()
    print("GPT:", gpt_reply)


In [16]:
for i in range(5):
    print(f"\n🔁 Round {i + 1}")
    conversation_round()


🔁 Round 1
🧠 Claude replying...
Claude: Well, I'm really curious about the basics of Python. I know a bit of JavaScript, but I want to understand how Python is different and what the core syntax and concepts are. Could you give me an overview of the basics? I'm eager to learn!
🔍 Gemini replying...
Gemini: Okay, awesome! I'm really excited to learn. I've heard a lot about Python being used for everything, but I'm still a bit fuzzy on the basics. I think I understand *variables* a little bit, like how you can store stuff like numbers or text in them. And I saw someone write a simple "Hello, world!" program once.

So, maybe we could start with... how do you actually *do* something with those variables? Like, how do you add two numbers together, or change some text? And what are all the different kinds of "stuff" you can store in a variable besides just numbers and text? Are there things like, I dunno, lists? I've heard of those! Also, what's the deal with "functions?" Are those like littl